# Data Loading and Preprocessing

* No data augmentation was used
* models evaluate the entire test directory
* adam optimiser
* 10 epochs only

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# path to dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

# # data generators
# train_datagen = ImageDataGenerator(rescale=1./255)
# validation_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# Set batch size and img size
batch_size = 64

# Load and preprocess the training dataset
train_data_v2M = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(260, 260),  #efficientnet- 260x260
    batch_size=batch_size,
)
train_data_x = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(299, 299),  # Xception requires input size of 299x299
    batch_size=batch_size  # Set class_mode according to your dataset
)

# Load and preprocess the validation dataset
validation_data_v2M = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(260, 260),
    batch_size=batch_size
)
validation_data_x = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(299, 299),
    batch_size=batch_size
)

test_data_v2M = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(260, 260),  #efficientnet- 260x260
    batch_size=batch_size,
)
test_data_x = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(299, 299),  # Xception requires input size of 299x299
    batch_size=batch_size  # Set class_mode according to your dataset
)

2024-06-27 17:33:05.189750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 17:33:05.189878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 17:33:05.326918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 140002 files belonging to 2 classes.
Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


# Xception

In [2]:
import keras
from keras.applications import Xception
from keras.optimizers import Adam
import torch
import pandas as pd

# Load the pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer for binary classification
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)  # 2 classes for binary classification

# Create the fine-tuned model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
with tf.device('/GPU:0'):
    history = model.fit(train_data_x, epochs=10, validation_data=validation_data_x)
    
model.save('xception.h5')

history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history.csv', index=False)


# Evaluate the model on the test set
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_data_x)
    print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10


I0000 00:00:1719510056.905575      76 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2188/2188 ━━━━━━━━━━━━━━━━━━━━ 697s 305ms/step - accuracy: 0.6255 - loss: 1.0015 - val_accuracy: 0.6827 - val_loss: 0.6424
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 649s 296ms/step - accuracy: 0.6882 - loss: 0.6417 - val_accuracy: 0.6988 - val_loss: 0.5954
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 649s 297ms/step - accuracy: 0.6974 - loss: 0.6264 - val_accuracy: 0.6912 - val_loss: 0.6045
Epoch 4/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 649s 296ms/step - accuracy: 0.6990 - loss: 0.6211 - val_accuracy: 0.6753 - val_loss: 0.6274
Epoch 5/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 647s 295ms/step - accuracy: 0.6960 - loss: 0.6369 - val_accuracy: 0.7075 - val_loss: 0.5992
Epoch 6/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 647s 295ms/step - accuracy: 0.7051 - loss: 0.6084 - val_accuracy: 0.6948 - val_loss: 0.6070
Epoch 7/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 647s 295ms/step - accuracy: 0.7008 - loss: 0.6170 - val_accuracy: 0.7070 - val_loss: 0.5941
Epoch 8/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 646s 295ms/step - accuracy: 0.7

# EfficientNetV2M

In [3]:
import tensorflow.keras as keras
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.optimizers import Adam

# Step 1: Load the EfficientNetV2-M model
model = EfficientNetV2M(weights='imagenet', input_shape=(260, 260, 3), include_top=False)

# Step 2: Freeze the convolutional base
model.trainable = False

# Step 3: Add custom classification head
x = keras.layers.GlobalAveragePooling2D()(model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# Step 4: Compile the model
model = keras.models.Model(inputs=model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
with tf.device('/GPU:0'):
    history = model.fit(train_data_v2M, epochs=10, validation_data=validation_data_v2M)

# Step 6: Evaluate the model
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_data_v2M)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 970s 384ms/step - accuracy: 0.7245 - loss: 0.5426 - val_accuracy: 0.7695 - val_loss: 0.4781
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 722s 330ms/step - accuracy: 0.7631 - loss: 0.4861 - val_accuracy: 0.7739 - val_loss: 0.4710
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 724s 331ms/step - accuracy: 0.7713 - loss: 0.4763 - val_accuracy: 0.7793 - val_loss: 0.4623
Epoch 4/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 719s 329ms/step - accuracy: 0.7738 - loss: 0.4701 - val_accuracy: 0.7815 - val_loss: 0.4583
Epoch 5/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 721s 329ms/step - accuracy: 0.7754 - loss: 0.4666 - val_accuracy: 0.7819 - val_loss: 0.4567
Epoch 6/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 721s 329ms/step - accuracy: 0.7775 - loss: 0.4639 - val_accuracy: 0.7818 - val_loss: 0.4565
Epoch 7/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 721s 330ms/step - accuracy: 0.7778 - loss: 0.4624 - val_accuracy: 0.7822 - val_loss: 0.4569
Epoch 8/1